In [1]:
from config import API_EODHD
import pandas as pd
import psycopg2
import numpy as np
import requests
from bs4 import BeautifulSoup

In [50]:
symbol = "AAPL"
type_ = "call"
strike_from = 150
sort = "exp_date"
url = f"https://eodhd.com/api/mp/unicornbay/options/eod?filter[underlying_symbol]={symbol}&filter[type]={type_}&filter[strike_from]={strike_from}&sort={sort}&api_token={API_EODHD}"
res = requests.get(url)

In [98]:
start_date = "2024-12-27"
end_date = "2024-12-27"
underlying_symbol = "AAPL"
url1 = f"https://eodhd.com/api/mp/unicornbay/options/eod?filter[underlying_symbol]={underlying_symbol}&filter[exp_date_from]={start_date}&filter[exp_date_to]={end_date}&sort=-exp_date&api_token={API_EODHD}"
res1 = requests.get(url1)
res_json1 = res1.json()
res_json1.keys()
data1 = res_json1['data']
attributes1 = [i['attributes'] for i in data1]
a1 = pd.DataFrame(attributes1)

In [2]:
# insert data
def insert_dataframe(cursor, conn, df, table_name="options_data"):

    # 替換 NaN 為 None，以便 PostgreSQL 可以正確處理 NULL 值
    df = df.where(pd.notnull(df), None)

    # 定義 SQL 插入語句
    insert_sql = f"""
    INSERT INTO {table_name} (
        contract, underlying_symbol, exp_date, expiration_type, type, strike,
        exchange, currency, open, high, low, last, last_size, change, pctchange, 
        previous, previous_date, bid, bid_date, bid_size, ask, ask_date, ask_size, 
        moneyness, volume, volume_change, volume_pctchange, open_interest, 
        open_interest_change, open_interest_pctchange, volatility, volatility_change, 
        volatility_pctchange, theoretical, delta, gamma, theta, vega, rho, tradetime, 
        vol_oi_ratio, dte, midpoint
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (contract) DO NOTHING;
    """  # `ON CONFLICT (contract) DO NOTHING` 避免插入重複數據

    # 批量插入數據
    data_tuples = [tuple(x) for x in df.to_numpy()]
    cursor.executemany(insert_sql, data_tuples)

    # 提交更改並關閉連接
    conn.commit()

In [3]:
class SymbolParams:
    def __init__(self, underlying_symbol, start_date, end_date, type_):
        self.underlying_symbol = underlying_symbol;
        self.start_date = start_date
        self.end_date = end_date
        self.type_ = type_
    def __str__(self):
        return f"SymbolParams(underlying_symbol={self.underlying_symbol}, start_date={self.start_date}, end_date={self.end_date}, type={self.type_})"


In [4]:
import yfinance as yf
from tqdm import tqdm
def calculate_price_at_certain_delta(a1: pd.DataFrame, da='2024-01-16', delta=0.2):
    s = a1[(a1['tradetime'] == da) & (a1['delta'].between(delta-0.19, delta+0.19))]
    if len(s) >= 1: pass
    higher = s[s['delta'] > delta]
    lower = s[s['delta'] < delta]
    if len(higher) == 0 or len(lower) == 0: return None;
    below = lower.sort_values(by='delta', ascending=False).iloc[0, :] # Max delta below 0.2
    above = higher.sort_values(by='delta', ascending=True).iloc[0, :]

    if below.empty or above.empty:
        return None
    else:
        # Linear interpolation for price at delta = 0.2
        x1, y1 = below['delta'], below['strike']
        x2, y2 = above['delta'], above['strike']
        price_at_02 = y1 + (delta - x1) * (y2 - y1) / (x2 - x1)
        return price_at_02
    
def calculate_daily_bias(a1, da, underlying_assets_stock_price=185.91):
    _p20 = calculate_price_at_certain_delta(a1, da, 0.2)
    _n20 = calculate_price_at_certain_delta(a1, da, -0.2)
    if not _p20 or not _n20: return None;
    ratio = (_p20 - underlying_assets_stock_price)/(underlying_assets_stock_price-_n20)
    return ratio

def get_delta_factors_in_year(underlying_symbol, date_end):
    # get conn
    DB_HOST = 'localhost'
    DB_NAME = 'us'
    DB_USER = 'postgres'
    DB_PASS = 'buddyrich134'
    def get_db_connection():
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        return conn
    conn = get_db_connection()
    cursor = conn.cursor()

    # get exp_dates
    # sqlStr = "SELECT exp_date from expiration_dates where exp_date >= '2024-01-01' and exp_date <= '2024-12-31'";
    sqlStr = f"SELECT exp_date from expiration_dates where exp_date >= '2024-01-01' and exp_date <= '{date_end}'";
    cursor.execute(sqlStr)
    conn.commit()
    exp_date_array_dt = cursor.fetchall()
    exp_date_array = np.array([i[0].strftime("%Y-%m-%d") for i in exp_date_array_dt])
    
    # get stock price
    df = yf.download(underlying_symbol, start='2023-12-01', progress=False)
    print(underlying_symbol, df.index[-1])
    date_factor_dict = {}
    
    for date in tqdm(exp_date_array[:]):
        symbol = SymbolParams(underlying_symbol, date, date, "call")
        url = f"https://eodhd.com/api/mp/unicornbay/options/eod?filter[underlying_symbol]={symbol.underlying_symbol}&filter[exp_date_from]={symbol.start_date}&filter[exp_date_to]={symbol.end_date}&sort=-exp_date&api_token={API_EODHD}"
        res = requests.get(url)
        res_json = res.json()
        res_json.keys()
        data = res_json['data']
        attributes = [i['attributes'] for i in data]
        a = pd.DataFrame(attributes)
        insert_dataframe(cursor, conn, a, table_name='options_data')
        tradetime_date_list = sorted(list(set(a['tradetime'].dropna())))
        for date in tradetime_date_list[:-1]:
            try:
                underlying_assets_stock_price = df.loc[:date].iloc[-2]['Close']
                factor = calculate_daily_bias(a, date, underlying_assets_stock_price)
                if factor is not None: date_factor_dict[date] = factor
            except:
                print(date, underlying_symbol)
                print(df.loc[:date])
    return date_factor_dict

In [5]:
def insert_delta_factor_into_db(cursor, conn, factor_dict, symbol):
    factor_name = "option_delta20_using_prev_close"
    # 插入數據
    insert_query = """
        INSERT INTO factors (trade_date, factor_name, symbol, factor)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (trade_date, factor_name, symbol)
        DO UPDATE SET factor = EXCLUDED.factor;
    """
    
    # 批量插入
    for date, factor_value in factor_dict.items():
        cursor.execute(insert_query, (date, factor_name, symbol, float(factor_value)))  # np.float64 轉 float
    conn.commit()
# get conn
DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute("SELECT distinct code from public.maincode where listed = 'us30';")
conn.commit()
code_list_30 = np.array([i[0] for i in cursor.fetchall() if i != 'DIA'])

In [6]:
for symbol in tqdm(code_list_30):
    date_factor_dict = get_delta_factors_in_year(symbol, date_end='2025-02-14')
    insert_delta_factor_into_db(cursor, conn, date_factor_dict, symbol)
    

  0%|          | 0/31 [00:00<?, ?it/s]

MMM 2025-02-07 00:00:00


  3%|▎         | 1/31 [01:53<56:33, 113.10s/it]

UNH 2025-02-07 00:00:00


  6%|▋         | 2/31 [03:46<54:36, 112.99s/it]

...

2022-11-30 V
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


 19%|█▉        | 6/31 [12:07<51:14, 122.99s/it]

TRV 2025-02-07 00:00:00


 19%|█▉        | 6/31 [12:08<50:33, 121.35s/it]


KeyError: 'tradetime'

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
df = yf.download("JNJ", start='2023-12-01', end="2024-12-31")
date_factor_pd = pd.DataFrame([date_factor_dict]).T
date_factor_pd.columns = ['signal']
date_factor_pd['price'] = df['Close'].loc[date_factor_pd.index[0]:date_factor_pd.index[-1]]
date_factor_pd['pct'] = np.log(date_factor_pd['price']/date_factor_pd['price'].shift(1)).shift(-1)
# date_factor_pd['strat'] = date_factor_pd.apply(lambda row: row['pct'] if row['signal'] > 1 else -row['pct'], axis=1)
date_factor_pd['strat'] = date_factor_pd.apply(lambda row: row['pct'] if row['signal'] > 1 else 0, axis=1)
date_factor_pd['strat_signal_1_0'] = date_factor_pd['signal'].apply(lambda x: 1 if x > 1 else 0)
# date_factor_pd['strat'].cumsum().plot()


fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles="GS")

date_factor_pd['diff'] = date_factor_pd['strat'].cumsum() - date_factor_pd['pct'].cumsum()
fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['pct'].cumsum(),
    mode='lines',
    name='bm'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['strat'].cumsum() * 0.95,
    mode='lines',
    name='strat'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['diff'],
    mode='lines',
    name='exceed return'
), row=1, col=1)


fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['strat_signal_1_0'],
    mode='lines',
    name='Signal'
), row=3, col=1)

fig.update_layout(
    title="GS",
    template="plotly_white",
    height=800,
    width=1200
)


In [ ]:
# CREATE TABLE factors (
#     trade_date DATE,
#     factor_name CHAR(50),
#     symbol CHAR(50),
#     factor FLOAT
    
# );
# ALTER TABLE factors ADD CONSTRAINT unique_trade_factor_symbol UNIQUE (trade_date, factor_name, symbol);

In [ ]:
# CREATE TABLE expiration_dates (
#     id SERIAL PRIMARY KEY,
#     exp_date DATE UNIQUE  -- Ensures unique expiration dates
# );


In [ ]:
# CREATE TABLE options_data (
#     contract TEXT PRIMARY KEY,         -- 合約名稱（設為 PRIMARY KEY）
#     underlying_symbol TEXT,            -- 標的代號
#     exp_date DATE,                     -- 到期日
#     expiration_type TEXT,               -- 到期類型
#     type TEXT,                          -- 選擇權類型（Call/Put）
#     strike DOUBLE PRECISION,            -- 履約價
#     exchange TEXT,                      -- 交易所
#     currency TEXT,                      -- 貨幣
#     open DOUBLE PRECISION,              -- 開盤價
#     high DOUBLE PRECISION,              -- 最高價
#     low DOUBLE PRECISION,               -- 最低價
#     last DOUBLE PRECISION,              -- 最新成交價
#     last_size BIGINT,                   -- 最新成交數量
#     change DOUBLE PRECISION,            -- 價格變動
#     pctchange DOUBLE PRECISION,         -- 價格變動百分比
#     previous DOUBLE PRECISION,          -- 前一交易日收盤價
#     previous_date DATE,                 -- 前一交易日日期（NULLABLE）
#     bid DOUBLE PRECISION,               -- 買價
#     bid_date TEXT,                      -- 買價更新時間（可以是TEXT）
#     bid_size BIGINT,                    -- 買單數量
#     ask DOUBLE PRECISION,               -- 賣價
#     ask_date TEXT,                      -- 賣價更新時間（可以是TEXT）
#     ask_size BIGINT,                    -- 賣單數量
#     moneyness DOUBLE PRECISION,         -- Moneyness 值
#     volume BIGINT,                      -- 成交量
#     volume_change BIGINT,               -- 成交量變動
#     volume_pctchange DOUBLE PRECISION,  -- 成交量變動百分比
#     open_interest BIGINT,               -- 未平倉量
#     open_interest_change BIGINT,        -- 未平倉量變動
#     open_interest_pctchange DOUBLE PRECISION, -- 未平倉量變動百分比
#     volatility DOUBLE PRECISION,        -- 隱含波動率
#     volatility_change DOUBLE PRECISION, -- 波動率變動
#     volatility_pctchange DOUBLE PRECISION, -- 波動率變動百分比
#     theoretical DOUBLE PRECISION,       -- 理論價格
#     delta DOUBLE PRECISION,             -- Delta
#     gamma DOUBLE PRECISION,             -- Gamma
#     theta DOUBLE PRECISION,             -- Theta
#     vega DOUBLE PRECISION,              -- Vega
#     rho DOUBLE PRECISION,               -- Rho
#     tradetime TEXT,                     -- 交易時間
#     vol_oi_ratio DOUBLE PRECISION,      -- 成交量 / 未平倉比
#     dte BIGINT,                         -- 到期天數
#     midpoint DOUBLE PRECISION           -- 買賣價中間值
# );

# ALTER TABLE options_data 
# ALTER COLUMN last_size TYPE DOUBLE PRECISION,
# ALTER COLUMN bid_size TYPE DOUBLE PRECISION,
# ALTER COLUMN ask_size TYPE DOUBLE PRECISION,
# ALTER COLUMN volume TYPE DOUBLE PRECISION,
# ALTER COLUMN volume_change TYPE DOUBLE PRECISION,
# ALTER COLUMN open_interest TYPE DOUBLE PRECISION,
# ALTER COLUMN open_interest_change TYPE DOUBLE PRECISION,
# ALTER COLUMN dte TYPE DOUBLE PRECISION;

In [ ]:
# functions to create database
DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn


conn = get_db_connection()
cursor = conn.cursor()

cursor.executemany(
    "INSERT INTO expiration_dates (exp_date) VALUES (%s) ON CONFLICT (exp_date) DO NOTHING",
    [(d,) for d in exp_date_array]
)
conn.commit()